In [1]:
%%capture

# get_corpus_path
# get_txt_orig_path
# get_txt_clean_path
%run ../path_manager.ipynb


# CorpusCleaner
%run ../DataCleanerModule.ipynb

## Jupyter.notebook.save_checkpoint()

In [2]:
# w = spacy.load('/R/spacy_data/en_core_web_sm/en_core_web_sm-2.0.0', disable=['parser', 'ner', 'textcat'])

# from spacy.lemmatizer import Lemmatizer
# from nltk.corpus import wordnet
# l = Lemmatizer()
# lm = l('substantially', 'ADV')

# l = WordNetLemmatizer()
# l.lemmatize('substantially', wordnet.VERB)
# [token.lemma_ for token in w('initial jobless claim fall low level result to substantially lower expectation time.')]

In [3]:
ENCHANT_INSTALLED

True

# Misspelling correction

In [4]:
CORPUS_ID = 'WB'

In [5]:
TXT_ORIG_DIR = get_txt_orig_path(CORPUS_ID)
# TXT_CLEAN_DIR = os.path.join(get_corpus_path(CORPUS_ID), 'TXT_PCLEAN')  # 
TXT_CLEAN_DIR = get_txt_clean_path(CORPUS_ID)

In [6]:
metadata_file = pd.read_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_metadata.csv'), low_memory=False)
metadata_file = metadata_file.dropna(axis=0, subset=['url_txt'])

In [7]:
metadata_file.shape

(246240, 29)

In [7]:
metadata_file.shape

(229927, 29)

In [8]:
metadata_file.head(2)

,corpus,id,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,...,adm_region,geo_region,country,wb_lending_instrument,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id
0,wb,wb_19431275,/NLP/CORPUS/WB/TXT_ORIG/wb_19431275.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_19431275.txt,RAD2066245936.txt,2014.0,Project Documents,Loan Agreement,NaN,NaN,...,Europe and Central Asia,NaN,Moldova,Development Policy Lending,IBRD/IDA,NaN,"Public sector governance,Rural development,Fin...",NaN,"Climate Change and Agriculture,Consumption,Fis...",P143283
1,wb,wb_19431317,/NLP/CORPUS/WB/TXT_ORIG/wb_19431317.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_19431317.txt,RAD1222573634.txt,2014.0,Project Documents,Financing Agreement,NaN,NaN,...,Europe and Central Asia,NaN,Moldova,Development Policy Lending,IBRD/IDA,NaN,"Public sector governance,Rural development,Fin...",NaN,"Climate Change and Agriculture,Consumption,Fis...",P143283


In [9]:
%%time
cleaner=ParallelCorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='../whitelists/whitelist_replacements_plurals_to_singular.csv',
    acronyms_file='../whitelists/whitelist_acronyms.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)

Doclen 18 < 50 = None
CPU times: user 119 ms, sys: 2.12 ms, total: 121 ms
Wall time: 120 ms


# Run with subset dataset

In [10]:
cleaner.spell_cache_dict.keys()

[]

In [11]:
os.path.join(TXT_ORIG_DIR, 'wb_19431317')

'/home/wb536061/wbes2474/NLP/CORPUS/WB/TXT_ORIG/wb_19431317'

In [ ]:
# filepath = os.path.join(TXT_ORIG_DIR, 'wb_19431317.txt')
# with open(filepath, 'rb') as fl:
#     # Use context so that the file will be closed automatically upon exit from the context.
#     text = fl.read()
#     text = text.decode('utf-8', errors='ignore')
#     text = text.lower()
    
# r = cleaner.clean_text(text, filen='wb_19431317')
# l = cleaner.lemmatize_text(text)
# print(r['exception'])
# lmtzr_spacy = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

In [ ]:
%%time
# # doclist=['436966.txt', '2002839.txt']

doclist = [f"{id}.txt" for id in metadata_file['id'].sample(n=len(metadata_file), random_state=1029)]

print(len(doclist))
# wb_8886341
print(pd.datetime.now())
output=cleaner.batch_clean_docs(
    doclist,
#     default_docs_per_worker=10,
    batch_size=96,  # x4 of num_workers
    save_docs=True,
    skip_existing=True,
    collect_spell_errors=False)
print(pd.datetime.now())

246240
2019-10-18 11:57:51.329434
Doclen 7 < 50 = wb_31422591
Doclen 5 < 50 = wb_24998598
Doclen 40 < 50 = wb_18225021
Docset 96
Empty doc post lemmatizer = wb_749898
Doclen 33 < 50 = wb_18231235
Doclen 25 < 50 = wb_18156363
Docset 192
Doclen 10 < 50 = wb_7764157
Doclen 16 < 50 = wb_18176503
Doclen 0 < 50 = wb_27622279
Docset 288
Doclen 15 < 50 = wb_19452972
Doclen 13 < 50 = wb_29757218
Doclen 8 < 50 = wb_29816810
Doclen 8 < 50 = wb_20358821


Process Process-335:
Traceback (most recent call last):
  File "/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-86cc4889d0c1>", line 269, in load_existing_and_extract_metadata
    predict_lang = detect_langs(text)[0]
  File "/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/langdetect/detector_factory.py", line 137, in detect_langs
    return detector.get_probabilities()
  File "/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/langdetect/detector.py", line 143, in get_probabilities
    self._detect_block()
  File "/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/langdetect/detector.py", line 150, in _detect_block
    raise LangDetectException(ErrorCode.CantDetectError, 'No features in text.')
langd

Doclen 3 < 50 = wb_23856498
Docset 384
Empty doc post lemmatizer = wb_25505027
Doclen 25 < 50 = wb_18196581
Doclen 5 < 50 = wb_30970502
Doclen 0 < 50 = wb_29251758
Docset 480
Doclen 17 < 50 = wb_17210551
Doclen 7 < 50 = wb_18149857
Doclen 9 < 50 = wb_18179555
Docset 576
Doclen 5 < 50 = wb_31113849
Doclen 13 < 50 = wb_27027184
Failed lemmatization for wb_12955973
Docset 672
Doclen 5 < 50 = wb_24332963
Doclen 0 < 50 = wb_28145402
Doclen 0 < 50 = wb_29278341
Doclen 31 < 50 = wb_732595
Docset 768
Doclen 7 < 50 = wb_30764928
Doclen 14 < 50 = wb_6376753
Doclen 5 < 50 = wb_31486404
Doclen 10 < 50 = wb_29723075
Doclen 3 < 50 = wb_24430305
Doclen 28 < 50 = wb_19893346
Docset 864
Doclen 9 < 50 = wb_18176530
Doclen 27 < 50 = wb_18156036
Failed lemmatization for wb_6431797
Doclen 21 < 50 = wb_23969498
Doclen 19 < 50 = wb_18150537
Doclen 8 < 50 = wb_18210104
Doclen 4 < 50 = wb_18661129
Doclen 5 < 50 = wb_31365546
Doclen 34 < 50 = wb_19893333
Docset 960
Doclen 23 < 50 = wb_18196884
Doclen 7 < 50 = w

### Convert the output into a dataframe for easier processing

In [14]:
misc_metadata = pd.DataFrame(output)
misc_metadata.shape

(244661, 7)

In [15]:
misc_metadata.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_derived_metadata.csv'))

In [16]:
misc_metadata.head(2)

,lang,tokens,text,spell_errors,skipped,exception,write_status
wb_31422591,"(ERROR, 0)",0,NaN,NaN,Doclen 7 < 50 = wb_31422591,None,False
wb_19200456,"(en, 0.9999953724491716)",385,NaN,NaN,,None,True


Log items that were skipped due to some unexpected errors

In [17]:
misc_metadata[misc_metadata.skipped != ''].skipped.to_csv(f'{CORPUS_ID.lower()}_skipped-items-{pd.datetime.now()}.log')

/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


Generate new attributes based on the `lang` column from `misc_metadata`.

In [18]:
misc_metadata['language_detected'] = misc_metadata.lang.map(lambda x: x[0])
misc_metadata['language_score'] = misc_metadata.lang.map(lambda x: x[1])
misc_metadata = misc_metadata.drop('lang', axis=1)

misc_metadata.index.name = 'id'
misc_metadata = misc_metadata.reset_index()

In [19]:
misc_metadata.head(2)

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,wb_31422591,0,NaN,NaN,Doclen 7 < 50 = wb_31422591,None,False,ERROR,0.000000
1,wb_19200456,385,NaN,NaN,,None,True,en,0.999995


In [20]:
misc_metadata[misc_metadata.skipped != ''].head()

,id,tokens,text,spell_errors,skipped,exception,write_status,language_detected,language_score
0,wb_31422591,0,NaN,NaN,Doclen 7 < 50 = wb_31422591,None,False,ERROR,0.000000
6,wb_24998598,0,NaN,NaN,Doclen 5 < 50 = wb_24998598,None,False,ERROR,0.000000
27,wb_18225021,0,NaN,NaN,Doclen 40 < 50 = wb_18225021,None,False,ERROR,0.000000
61,wb_749898,0,NaN,NaN,Empty doc post lemmatizer = wb_749898,None,False,ERROR,0.000000
72,wb_29562458,0,NaN,NaN,Not in english | ca:0.9999942797661113,None,False,ca,0.999994


In [21]:
misc_metadata['id'].nunique(), misc_metadata.shape

(244661, (244661, 9))

In [22]:
valid_processed_ids = misc_metadata[(misc_metadata.skipped == '') & (misc_metadata.write_status == True)].set_index('id').index
len(valid_processed_ids)

217949

In [23]:
complete_metadata_file = pd.concat([metadata_file.drop_duplicates('id').set_index('id'), misc_metadata.set_index('id')[['language_detected', 'language_score', 'tokens', 'write_status']]], axis=1)
complete_metadata_file = complete_metadata_file.loc[valid_processed_ids]

/home/wb536061/anaconda3/envs/wb-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [24]:
complete_metadata_file.shape

(217949, 32)

In [25]:
complete_metadata_file.head(2)

,corpus,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,title,...,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id,language_detected,language_score,tokens,write_status
id,,,,,,,,,,,,,,,,,,,,,
wb_19200456,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_19200456.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_19200456.txt,RAD639317460.txt,2014.0,Project Documents,Agreement,NaN,NaN,Official Documents - Amendment to the Suppleme...,...,IBRD/IDA,"Public sector governance,Rural development,Tra...","Public sector governance,Rural services and in...",NaN,"Rural Roads & Transport,Disease Control & Prev...",P094488,en,0.999995,385.0,True
wb_6448332,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_6448332.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_6448332.txt,E12530EMP1Land0Registry.txt,2005.0,Project Documents,Environmental Assessment,NaN,NaN,Bosnia and Herzegovina - Land Registration Pro...,...,IBRD/IDA,NaN,NaN,"Agriculture, Fishing and Forestry|Financial Se...","Health Care Services Industry,Urban Solid Wast...",P096200,en,0.999996,2451.0,True


In [26]:
complete_metadata_file.to_csv(os.path.join(get_corpus_path(CORPUS_ID), f'{CORPUS_ID.lower()}_metadata_complete.csv'))
complete_metadata_file.head(2)

,corpus,path_original,path_clean,filename_original,year,major_doc_type,doc_type,author,collection,title,...,wb_product_line,wb_major_theme,wb_theme,wb_sector,wb_subtopic_src,wb_project_id,language_detected,language_score,tokens,write_status
id,,,,,,,,,,,,,,,,,,,,,
wb_19200456,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_19200456.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_19200456.txt,RAD639317460.txt,2014.0,Project Documents,Agreement,NaN,NaN,Official Documents - Amendment to the Suppleme...,...,IBRD/IDA,"Public sector governance,Rural development,Tra...","Public sector governance,Rural services and in...",NaN,"Rural Roads & Transport,Disease Control & Prev...",P094488,en,0.999995,385.0,True
wb_6448332,wb,/NLP/CORPUS/WB/TXT_ORIG/wb_6448332.txt,/NLP/CORPUS/WB/TXT_CLEAN/wb_6448332.txt,E12530EMP1Land0Registry.txt,2005.0,Project Documents,Environmental Assessment,NaN,NaN,Bosnia and Herzegovina - Land Registration Pro...,...,IBRD/IDA,NaN,NaN,"Agriculture, Fishing and Forestry|Financial Se...","Health Care Services Industry,Urban Solid Wast...",P096200,en,0.999996,2451.0,True


In [27]:
# Processing output

In [28]:
s = '''%%time
cleaner = CorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='whitelist_replacements_plurals_to_singular.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)'''

In [29]:
%pprint
print(s)

Pretty printing has been turned OFF
%%time
cleaner = CorpusCleaner(
    use_spellchecker=True, use_respeller=True, use_lemmatizer=True, use_spacy=True,
    replacements_plurals_to_singular_file='whitelist_replacements_plurals_to_singular.csv',
)

cleaner.set_input_folder(TXT_ORIG_DIR)
cleaner.set_output_folder(TXT_CLEAN_DIR)
